In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler,  Imputer
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df = pd.read_csv('data/vk_users_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35517 entries, 0 to 35516
Data columns (total 18 columns):
political     5400 non-null float64
country       34114 non-null object
smoking       6264 non-null float64
sex           35517 non-null int64
id            35517 non-null int64
last_name     35517 non-null object
alcohol       6189 non-null float64
religion      6357 non-null object
langs         4624 non-null object
city          32524 non-null object
relation      14826 non-null float64
age           35517 non-null float64
verified      35517 non-null int64
bdate         29173 non-null object
first_name    35517 non-null object
university    14826 non-null float64
life_main     6239 non-null float64
posts         35517 non-null object
dtypes: float64(7), int64(3), object(8)
memory usage: 4.9+ MB


In [3]:
# заменим не указанные значения категориальных признаков на -1
df = df.fillna(-1)
# В столбце university содержится id университета по БД VK. 
# Будем считать, что если он указан, то у пользователя есть высшее 
# образование, иначе нет (хотя строго это не так, он может быть просто 
# не указан. Но примем такое предположение)
df['high_education'] = df['university'].apply(lambda x: 0 if x < -0.5 else 1)
df.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education
0,-1.00,Россия,-1.00,1,2615791,Третьякова,-1.00,-1,-1,Москва,-1.00,26.00,0,5.3.1992,Анастасия,-1.00,-1.00,🌸🌸🌸 #id2615791 (fashionlioness) #модель #фотос...,0
1,-1.00,Россия,-1.00,2,148071868,Дмитриев,-1.00,-1,-1,Кострома,-1.00,18.00,0,-1,Сергей,-1.00,-1.00,Карантин)!!!!!!!!!,0
2,3.00,Россия,4.00,1,54774632,Власова,4.00,Православие,-1,Пермь,4.00,110.00,0,6.7,Анюта,0.00,1.00,"Не важно, сколько дверей закроется перед твоим...",1
3,-1.00,Россия,-1.00,1,76303980,Шабалкова,-1.00,-1,-1,Санкт-Петербург,-1.00,90.00,0,3.9,Анастасия,-1.00,-1.00,Друзья! Я собираю большую посылку с помощью д...,0
4,-1.00,Россия,1.00,2,104199626,Блейх,1.00,Православие,"['Русский', 'English']",Санкт-Петербург,1.00,26.00,0,6.11,Эдгар,1.00,1.00,Не мой среди твоих\nИстинный ариец. Характер —...,1


In [4]:
df_train, df_test = train_test_split(df, test_size=0.3)

In [5]:
df_train.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education
16905,-1.00,Россия,-1.00,1,289307299,Савельева,-1.00,-1,-1,Воронеж,5.00,14.00,0,3.6,Юлия,0.00,-1.00,Мешок с детдомом едет к вам \nЗа Юлю я пизды в...,1
19568,-1.00,Россия,-1.00,1,244128580,Оствальд,-1.00,-1,-1,Ростов-на-Дону,-1.00,102.00,0,-1,Натали,-1.00,-1.00,"DJ Алексей зажигает в клубе ""клуб"" с треком ""И...",0
7387,-1.00,Беларусь,4.00,2,159499195,Жогло,4.00,-1,-1,Минск,0.00,14.00,0,5.2.2002,Влад,0.00,-1.00,"Мммм\nВсе считают нас идеальными, ⚕\nПрошу, не...",1
31135,-1.00,Казахстан,-1.00,1,156680455,Гарыгина,-1.00,-1,"['Русский', 'English', 'Français']",Астана,0.00,106.00,0,4.4,Наталья,0.00,-1.00,😻😻😻✨\n♥♥♥C Днем Рождения!♥♥♥\nС Днем Рождения ...,1
35102,4.00,Россия,-1.00,1,5285020,Тимошкина,-1.00,в поиске,-1,Санкт-Петербург,4.00,94.00,0,26.3.1921,Дарья,1.00,-1.00,"Дашенька, С Новым Годом!\n😉 Открытки и поздрав...",1


In [6]:
df_test.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education
3450,-1.00,-1,-1.00,2,347347329,Сайтиев,-1.00,-1,-1,-1,-1.00,42.00,1,11.3.1975,Бувайсар,-1.00,-1.00,"😌\nПолагаю, для многих дагестанцев имя Ильмана...",0
7746,4.00,Россия,2.00,1,248026584,Фомина,2.00,Православие,-1,Москва,4.00,30.00,0,16.6.1987,Анастасия,0.00,6.00,ПОСТАВЬ ❤\nНАБЕРЕМ 100 ❤? \nСЛАБО?\nНастроение...,1
5087,-1.00,Россия,-1.00,2,11498757,Добровольский,-1.00,-1,-1,Москва,-1.00,22.00,0,24.9.1994,Илья,-1.00,-1.00,"О да) спасибо, Диме Найту за отличный подарок ...",0
11883,-1.00,Украина,-1.00,2,150569975,Лахман,-1.00,-1,-1,-1,-1.00,66.00,0,27.11,Валік,-1.00,-1.00,#бро,0
17452,-1.00,Казахстан,-1.00,1,213858335,Абдуова,-1.00,-1,-1,-1,-1.00,14.00,0,-1,Толкын,-1.00,-1.00,"🖤🔒\nДруг. \nКто такой друг, трудно объяснить. ...",0


Смысл значений в столбцах sex, political, smoking, alcohol, relation, life_main следующий (из описания API VK):

1) sex - пол пользователя:
1 - женский;
2 - мужской;
0 - пол не указан

2) political - политические предпочтения:

1 - коммунистические;
2 - социалистические;
3 - умеренные;
4 - либеральные;
5 - консервативные;
6 - монархические;
7 - ультраконсервативные;
8 - индиффирентные;
9 - либертарианские;

3) smoking, alcohol - отношение к курению, алкоголю:

1 - резко негативное;
2 - негативное;
3 - компромиссное;
4 - нейтральное;
5 - положительное;

4) relation - семейное положение:

1 - не женат/не замужем;
2 - есть друг/есть подруга;
3 - помолвлен/помолвлена;
4 - женат/замужем;
5 - всё сложно;
6 - в активном поиске;
7 - влюблён/влюблена;
8 - в гражданском браке;
0 - не указано;

5) life_main - главное в жизни:

1 - семья и дети;
2 - карьера и деньги;
3 - развлечения и отдых;
4 - наука и исследования;
5 - совершенствование мира;
6 - саморазвитие;
7 - красота и искусство;
8 - слава и влияние

Попытаемся предсказать возраст (по данным VK (скорее всего, может быть с ошибками: неизвестно, как он там определяется)) по постам пользователя, его полу (пол указан всегда) и указанным признаками из раздела personal, а также по наличию высшего образования

#### Функции преобразования постов пользователя

In [7]:
from nltk.corpus import stopwords

# используем для лемматизации библиотеку pymorphy2. Она работает с русским языком
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

stopwords_en = stopwords.words('english')
stopwords_ru = stopwords.words('russian')
stopwords_ge = stopwords.words('german')

stopwords_all = stopwords_en + stopwords_ru + stopwords_ge

In [19]:
def lemmatization(text):
    return morph.parse(text)[0].normal_form

def tokenizer(text):
    text = text.decode('utf-8').lower()
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub(ur'[\W]+', ' ', text, flags=re.U) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stopwords_all]
    tokenized = [re.sub(ur"\W", "", w, flags=re.U) for w in tokenized]
    tokenized = [lemmatization(w) for w in tokenized]
    return tokenized

In [32]:
def get_posts_col(df):
    return df[['posts']]

def get_categorial_cols(df):
    return df[['sex', 'high_education', 'political', 'smoking', 'alcohol', 'relation', 'life_main']]

vec = make_union(*[
    make_pipeline(FunctionTransformer(get_categorial_cols, validate=False), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_posts_col, validate=False))
])

In [33]:
x_train = vec.fit_transform(df_train)
x_train.shape

(24861, 8)

In [41]:
get_posts_col(df_test)

,posts
12562,С Новым годом Украинооо))\nТвоя школа...\n)\nО...
18369,7 миллиардов людей\n\n14 миллиардов лиц 🎭\n💎\n...
12482,...
23025,Новый кайфовый. Всем любовь.♥ #2018\n\n \nКриз...
29320,т.т\nЖдём)\n#Спасибо_Терри .третье место\nНам ...
31716,"Счастья!!!\nС Днем Рождения, сестра!!!\n💋 И сн..."
20173,❌💛\nВикинг 💪🏼\n🌹
19033,https://vk.com/avacs6\nhttps://vk.com/wall-105...
33922,Посмотри своих поклонников здесь : \n✨ ⭐ ⭐ ► ►...
5867,[id204530265|ван лав] * 😍\nНа этой планете сущ...
